# Use SageMaker Distributed Model Parallel with Amazon SageMaker to Launch an MNIST Training Job with Model Parallelization

SageMaker Distributed Model Parallel (SMP) is a model parallelism library for training large deep learning models that were previously difficult to train due to GPU memory limitations. SageMaker Distributed Model Parallel automatically and efficiently splits a model across multiple GPUs and instances and coordinates model training, allowing you to increase prediction accuracy by creating larger models with more parameters.

Use this notebook to configure Sagemaker Distributed Model Parallel to train a model using an example PyTorch training script, `utils/pt_mnist.py` and [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#train-a-model-with-the-sagemaker-python-sdk). 


### Additional Resources

If you are a new user of Amazon SageMaker, you may find the following helpful to learn more about SMP and using SageMaker with Pytorch. 

* To learn more about the SageMaker model parallelism library, see [Model Parallel Distributed Training with SageMaker Distributed](http://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html).

* To learn more about using the SageMaker Python SDK with Pytorch, see [Using PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html).

* To learn more about launching a training job in Amazon SageMaker with your own training image, see [Use Your Own Training Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html).

## Amazon SageMaker Initialization

Run the following cells to initialize the notebook instance. Get the SageMaker execution role used to run this notebook.

In [1]:
pip install sagemaker-experiments

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sagemaker --upgrade

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%time
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import boto3
from time import gmtime, strftime

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

session = boto3.session.Session()

SageMaker Execution Role:arn:aws:iam::230755935769:role/SageMakerExecutionRoleMLOps
CPU times: user 1.01 s, sys: 427 ms, total: 1.44 s
Wall time: 938 ms


## Prepare your training script

Run the following cell to view an example-training script you will use in this demo. This is a PyTorch 1.6 trianing script that uses the MNIST dataset. 

You will see that the script contains `SMP` specific operations and decorators, which configure model parallel training. See the training script comments to learn more about the SMP functions and types used in the script.

In [4]:
# Run this cell to see an example of a training scripts that you can use to configure -
# SageMaker Distributed Model Parallel with PyTorch version 1.6
!cat utils/pt_mnist.py

# Future
from __future__ import print_function

# Standard Library
import os, time
import argparse
import math
import random

# Third Party
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.optim.lr_scheduler import StepLR
from torchnet.dataset import SplitDataset
from torchvision import datasets, transforms

# First Party
import smdistributed.modelparallel.torch as smp

# SM Distributed: import scaler from smdistributed.modelparallel.torch.amp, instead of torch.cuda.amp

# Make cudnn deterministic in order to get the same losses across runs.
# The following two lines can be removed if they cause a performance impact.
# For more details, see:
# https://pytorch.org/docs/stable/notes/randomness.html#cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


def aws_s3_sync(source, destination):
    
    """aws s3 sync in quiet mode and time profile"

## Define SageMaker Training Job

Next, you will use SageMaker Estimator API to define a SageMaker Training Job. You will use an [`Estimator`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to define the number and type of EC2 instances Amazon SageMaker uses for training, as well as the size of the volume attached to those instances. 

You can update the following:
* `processes_per_host`
* `entry_point`
* `instance_count`
* `instance_type`
* `base_job_name`

In addition, you can supply and modify configuration parameters for the SageMaker Distributed Model Parallel library. These parameters will be passed in through the `distributions` argument, as shown below.

### Update the Type and Number of EC2 Instances Used

Specify your `processes_per_host`. Note that it must be a multiple of your partitions, which by default is 2.

The instance type and number of instances you specify in `instance_type` and `instance_count` respectively will determine the number of GPUs Amazon SageMaker uses during training. Explicitly, `instance_type` will determine the number of GPUs on a single instance and that number will be multiplied by `instance_count`. 

You must specify values for `instance_type` and `instance_count` so that the total number of GPUs available for training is equal to `partitions` in `config` of `smp.init` in your training script. 


To look up instances types, see [Amazon EC2 Instance Types](https://aws.amazon.com/sagemaker/pricing/).


### Uploading Checkpoint During Training or Resuming Checkpoint from Previous Training
We also provide a custom way for users to upload checkpoints during training or resume checkpoints from previous training. We have integrated this into our `pt_mnist.py` example script. Please see the functions `aws_s3_sync`, `sync_local_checkpoints_to_s3`, and `sync_s3_checkpoints_to_local`. For the purpose of this example, we are only uploading a checkpoint during training, by using `sync_local_checkpoints_to_s3`. 


After you have updated `entry_point`, `instance_count`, `instance_type` and `base_job_name`, run the following to create an estimator. 

In [5]:
sagemaker_session = sagemaker.session.Session(boto_session=session)
mpioptions = "-verbose -x orte_base_help_aggregate=0 "

all_experiment_names = [exp.experiment_name for exp in Experiment.list()]

#choose an experiment name (only need to create it once)
experiment_name = "SM-MP-DEMO"

# Load the experiment if it exists, otherwise create 
if experiment_name not in all_experiment_names:
    customer_churn_experiment = Experiment.create(
        experiment_name=experiment_name, sagemaker_boto_client=boto3.client("sagemaker")
    )
else:
    customer_churn_experiment = Experiment.load(
        experiment_name=experiment_name, sagemaker_boto_client=boto3.client("sagemaker")
    )

# Create a trial for the current run
trial = Trial.create(
        trial_name="SMD-MP-demo-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())),
        experiment_name=customer_churn_experiment.experiment_name,
        sagemaker_boto_client=boto3.client("sagemaker"),
    )


smd_mp_estimator = PyTorch(
          entry_point="pt_mnist.py", # Pick your train script
          source_dir='utils',
          role=role,
          instance_type='ml.p3.16xlarge',
          sagemaker_session=sagemaker_session,
          framework_version='1.6.0',
          py_version='py36',
          instance_count=1,
          distribution={
              "smdistributed": {
                  "modelparallel": {
                      "enabled":True,
                      "parameters": {
                          "microbatches": 4,
                          "placement_strategy": "spread",
                          "pipeline": "interleaved",
                          "optimize": "speed",
                          "partitions": 2,
                          "ddp": True,
                      }
                  }
              },
              "mpi": {
                    "enabled": True,
                    "processes_per_host": 2, # Pick your processes_per_host
                    "custom_mpi_options": mpioptions 
              },
          },
          base_job_name="SMD-MP-demo",
      )


Finally, you will use the estimator to launch the SageMaker training job.

In [ ]:
smd_mp_estimator.fit(
        experiment_config={
            "ExperimentName": customer_churn_experiment.experiment_name,
            "TrialName": trial.trial_name,
            "TrialComponentDisplayName": "Training",
        })

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: SMD-MP-demo-2021-01-18-05-08-17-225


2021-01-18 05:08:17 Starting - Starting the training job...
2021-01-18 05:08:42 Starting - Launching requested ML instancesProfilerReport-1610946497: InProgress
.........
2021-01-18 05:10:16 Starting - Preparing the instances for training.........
2021-01-18 05:11:44 Downloading - Downloading input data...
2021-01-18 05:12:04 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-18 05:14:56,727 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-01-18 05:14:56,807 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-18 05:14:58,236 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-18 05:14:58,689 sagemaker-training-toolkit INFO     Starting MPI run as worker node.
2021-01-18 05:14:58,689 sagemaker-training-toolkit INFO     Creating SSH daemon.


2021-01-18 05:15:07 Training - Training image download completed. Training in progress.[1,0]<stdout>:[2021-01-18 05:15:13.064: I smdistributed/modelparallel/torch/module_partition.py:274] Partition assignments:
[1,0]<stdout>:[2021-01-18 05:15:13.064: I smdistributed/modelparallel/torch/module_partition.py:278] main: 0
[1,0]<stdout>:[2021-01-18 05:15:13.064: I smdistributed/modelparallel/torch/module_partition.py:278] main/module: 0
[1,0]<stdout>:[2021-01-18 05:15:13.064: I smdistributed/modelparallel/torch/module_partition.py:278] main/module/net1: 0
[1,0]<stdout>:[2021-01-18 05:15:13.064: I smdistributed/modelparallel/torch/module_partition.py:278] main/module/net2: 1
[1,0]<stdout>:[2021-01-18 05:15:13.067: I smdistributed/modelparallel/torch/model.py:324] Finished partitioning the model
[1,0]<stdout>:[2021-01-18 05:15:13.067: I smdistributed/modelparallel/torch/model.py:280] Number of parameters on partition 0 are 4. 4 require grads
[1,0]<stdout>:algo-1:32:32 [0] NCCL INFO Bootstrap

[1,0]<stdout>:Train Epoch: 1 [51200/60000 (85%)]#011Loss: 0.122801
[1,0]<stdout>:Train Epoch: 1 [51840/60000 (86%)]#011Loss: 0.017895
[1,0]<stdout>:Train Epoch: 1 [52480/60000 (87%)]#011Loss: 0.026668
[1,0]<stdout>:Train Epoch: 1 [53120/60000 (88%)]#011Loss: 0.229757
[1,0]<stdout>:Train Epoch: 1 [53760/60000 (90%)]#011Loss: 0.141660
[1,0]<stdout>:Train Epoch: 1 [54400/60000 (91%)]#011Loss: 0.024510
[1,0]<stdout>:Train Epoch: 1 [55040/60000 (92%)]#011Loss: 0.004486
[1,0]<stdout>:Train Epoch: 1 [55680/60000 (93%)]#011Loss: 0.053406
[1,0]<stdout>:Train Epoch: 1 [56320/60000 (94%)]#011Loss: 0.079183
[1,0]<stdout>:Train Epoch: 1 [56960/60000 (95%)]#011Loss: 0.070476
[1,0]<stdout>:Train Epoch: 1 [57600/60000 (96%)]#011Loss: 0.108503
[1,0]<stdout>:Train Epoch: 1 [58240/60000 (97%)]#011Loss: 0.024963
[1,0]<stdout>:Train Epoch: 1 [58880/60000 (98%)]#011Loss: 0.008600
[1,0]<stdout>:Train Epoch: 1 [59520/60000 (99%)]#011Loss: 0.000265
